# Домашнее задание. Урок 4.

## Практическое задание

Вариант 1. (простой)

- обучить сверточную нейронную сеть в стиле AlexNet (с падением размера ядра свертки и последовательностью блоков свертка-пулинг  (conv-pool)-(conv-pool)-...) на датасете fashion-mnist или cifar10
- оценить рост точности при увеличении ширины сети (больше ядер)
- оценить рост точности при увеличении глубины сети (больше слоев)
- (опционально)сравнить с точностью полносвязной сети для этой выборки

## Особенности сети в стиле AlexNet

1. Последовательное уменьшение размерности ядра свертки
2. Последовательность блокок свертка-пулинг (conv-pool)-(conv-pool)

![](https://github.com/alex-chin/intro_neural_network/blob/main/homeworks/home04/AlexNet.png?raw=1)

In [6]:
from __future__ import print_function
import tensorflow.keras as keras # расскоментируйте эту строку, чтобы начать обучение
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import os

In [22]:
import pandas as pd

In [2]:
classes = ['самолет', 'автомобиль', 'птица', 'кот', 'олень', 'собака', 'лягушка', 'лошадь', 'корабль', 'грузовик']


## Установка параметров нейросети

In [16]:
batch_size = 32
num_classes = 10
epochs = 10
data_augmentation = False
num_predictions = 20
# save_dir = os.path.join(os.getcwd(), 'saved_models')
# model_name = 'keras_cifar10_trained_model.h5'


## Разделение тренировочной и тестовой выборки

In [17]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'тренировочные примеры')
print(x_test.shape[0], 'тестовые примеры')

x_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры


## Преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1

In [18]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255



## Конфигурирование нейросети

In [19]:
def bild_model(koef_core = 1, num_layers = 1):
    model = Sequential()

    # слои нейросети отвественные за свертку и max-pooling
    model.add(Conv2D(32 * koef_core, (3, 3), padding='same', input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))

    for _ in range(num_layers):
        model.add(Conv2D(32 * koef_core, (3, 3)))
        model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64 * koef_core, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64 * koef_core, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Dropout(0.25))

    # полносвязные слои нейронной сети
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    # компиляция модели
    model.compile(loss='categorical_crossentropy',
                  optimizer='SGD',
                  metrics=['accuracy'])

    return model

In [20]:
result = []
for koef_core in [1, 2]:
  for num_layers in [1,2]:
    model = bild_model(koef_core=koef_core, num_layers=num_layers)
    history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
    scores = model.evaluate(x_test, y_test, verbose=1)
    result.append([koef_core, num_layers, history.history['loss'][-1], history.history['accuracy'][-1], scores[0], scores[1]])

Epoch 1/10
1563/1563 [==============================] - 9s 5ms/step - loss: 2.0889 - accuracy: 0.2223 - val_loss: 1.8656 - val_accuracy: 0.3490
Epoch 2/10
1563/1563 [==============================] - 8s 5ms/step - loss: 1.8152 - accuracy: 0.3416 - val_loss: 1.6886 - val_accuracy: 0.3883
Epoch 3/10
1563/1563 [==============================] - 8s 5ms/step - loss: 1.5858 - accuracy: 0.4212 - val_loss: 1.4125 - val_accuracy: 0.4901
Epoch 4/10
1563/1563 [==============================] - 8s 5ms/step - loss: 1.4656 - accuracy: 0.4676 - val_loss: 1.3531 - val_accuracy: 0.5066
Epoch 5/10
1563/1563 [==============================] - 7s 5ms/step - loss: 1.3752 - accuracy: 0.5043 - val_loss: 1.2476 - val_accuracy: 0.5578
Epoch 6/10
1563/1563 [==============================] - 7s 5ms/step - loss: 1.3001 - accuracy: 0.5355 - val_loss: 1.2071 - val_accuracy: 0.5668
Epoch 7/10
1563/1563 [==============================] - 8s 5ms/step - loss: 1.2454 - accuracy: 0.5572 - val_loss: 1.1244 - val_accuracy:

In [23]:
# проверка работы обученной модели
pd.DataFrame(result, columns=['core', 'layers', 'loss train','accuracy train','loss test','accuracy test'])

,core,layers,loss train,accuracy train,loss test,accuracy test
0,1,1,1.104780,0.60790,1.060724,0.6302
1,1,2,1.097117,0.60872,1.036278,0.6373
2,2,1,0.987845,0.65122,0.938222,0.6711
3,2,2,0.992726,0.64874,0.928064,0.6670
